# Parte 7 - Análise de Emoção

In [ ]:
from transformers import pipeline
import pandas as pd
import torch

hinos_analise: pd.DataFrame = pd.read_pickle("..\\assets\\hinos_analise_tokens.pkl")

# Carregar modelo multilíngue de emoções
model_name = "pysentimiento/bert-pt-emotion"
classifier = pipeline(
    "text-classification",
    model=model_name,
    tokenizer=model_name,
    top_k=None,  # Substitui return_all_scores=True (depreciado)
    max_length=512,  # Limite máximo de tokens do BERT
    truncation=True,  # Trunca textos longos automaticamente
    device=-1  # Força uso da CPU para evitar problemas de memória
)

# Função para truncar texto se necessário
def truncar_texto(texto, max_tokens=400):
    """Trunca o texto para não exceder o limite do modelo"""
    if not isinstance(texto, str):
        return ""
    
    # Tokeniza e verifica o tamanho
    tokens = classifier.tokenizer.encode(texto, add_special_tokens=True, max_length=max_tokens, truncation=True)
    # Decodifica os tokens truncados
    texto_truncado = classifier.tokenizer.decode(tokens, skip_special_tokens=True)
    return texto_truncado

# Classificar emoções com tratamento de erro
def analisar_emocoes(texto):
    try:
        if not isinstance(texto, str) or len(texto.strip()) == 0:
            return {}
        
        # Truncar texto se necessário
        texto_processado = truncar_texto(texto)
        
        # Classificar emoções
        resultado = classifier(texto_processado)
        
        # O resultado é uma lista de listas, pegamos o primeiro elemento
        if isinstance(resultado, list) and len(resultado) > 0:
            emocoes_lista = resultado[0]  # Primeira (e única) amostra
            if isinstance(emocoes_lista, list):
                # Converter lista de dicts em dict plano {emoção: score}
                return {r["label"]: r["score"] for r in emocoes_lista}
        
        return {}
    
    except Exception as e:
        print(f"Erro ao processar texto: {str(e)[:100]}...")
        return {}

# Verificar alguns textos antes de processar tudo
print("Verificando tamanhos dos textos...")
print(f"Total de hinos: {len(hinos_analise)}")

# Testar com um texto específico primeiro
print("\nTestando modelo com um texto simples...")
texto_teste = "Jesus é o meu salvador e eu o amo muito"
resultado_teste = analisar_emocoes(texto_teste)
print(f"Resultado do teste: {list(resultado_teste.keys())[:5]}")  # Primeiras 5 emoções

# Processar apenas uma amostra primeiro para testar
print("\nTestando com uma amostra pequena...")
amostra = hinos_analise.head(3).copy()
amostra["emocoes"] = amostra["texto_limpo"].apply(analisar_emocoes)

print("Teste bem-sucedido! Resultado da amostra:")
for idx, row in amostra.iterrows():
    emocoes = row["emocoes"]
    if emocoes:
        top_emocao = max(emocoes.items(), key=lambda x: x[1])
        print(f"{row['nome'][:30]}... -> {top_emocao[0]}: {top_emocao[1]:.3f}")
    else:
        print(f"{row['nome'][:30]}... -> Erro no processamento")

print(f"\nPronto! Agora você pode processar todos os {len(hinos_analise)} hinos.")

In [ ]:
# Processar todos os hinos (pode demorar alguns minutos)
print("Processando análise de emoções para todos os hinos...")
print("Isso pode levar alguns minutos. Progresso:")

import time
start_time = time.time()

# Processar em lotes para mostrar progresso
batch_size = 50
total_batches = len(hinos_analise) // batch_size + 1

all_emotions = []
for i in range(0, len(hinos_analise), batch_size):
    batch = hinos_analise.iloc[i:i+batch_size]
    batch_emotions = batch["texto_limpo"].apply(analisar_emocoes)
    all_emotions.extend(batch_emotions.tolist())
    
    current_batch = i // batch_size + 1
    print(f"Lote {current_batch}/{total_batches} concluído ({i+len(batch)}/{len(hinos_analise)} hinos)")

# Adicionar resultados ao dataframe
hinos_analise["emocoes"] = all_emotions

end_time = time.time()
print(f"\nProcessamento concluído em {end_time - start_time:.1f} segundos!")
print(f"Total de hinos processados: {len(hinos_analise)}")

# Salvar resultados
hinos_analise.to_pickle("..\\assets\\hinos_analise_com_emocoes.pkl")
print("Resultados salvos em: ..\\assets\\hinos_analise_com_emocoes.pkl")

# Mostrar amostra dos resultados
print("\nAmostra dos resultados:")
for idx, row in hinos_analise.head(10).iterrows():
    emocoes = row["emocoes"]
    if emocoes:
        top_emocao = max(emocoes.items(), key=lambda x: x[1])
        print(f"{row['nome'][:40]}... -> {top_emocao[0]}: {top_emocao[1]:.3f}")
    else:
        print(f"{row['nome'][:40]}... -> Erro no processamento")

In [ ]:
# Análise e visualização dos resultados de emoções
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Verificar se os dados foram processados
if "emocoes" not in hinos_analise.columns:
    print("Execute a célula anterior primeiro para processar as emoções!")
else:
    print("Analisando resultados das emoções...")
    
    # Extrair a emoção dominante de cada hino
    emocoes_dominantes = []
    scores_dominantes = []
    
    for emocoes in hinos_analise["emocoes"]:
        if emocoes:
            top_emocao = max(emocoes.items(), key=lambda x: x[1])
            emocoes_dominantes.append(top_emocao[0])
            scores_dominantes.append(top_emocao[1])
        else:
            emocoes_dominantes.append("unknown")
            scores_dominantes.append(0.0)
    
    hinos_analise["emocao_dominante"] = emocoes_dominantes
    hinos_analise["score_dominante"] = scores_dominantes
    
    # 1. Distribuição das emoções dominantes
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    emocao_counts = pd.Series(emocoes_dominantes).value_counts().head(10)
    emocao_counts.plot(kind='bar', color='skyblue')
    plt.title('Top 10 Emoções Dominantes nos Hinos')
    plt.xlabel('Emoção')
    plt.ylabel('Quantidade de Hinos')
    plt.xticks(rotation=45)
    
    # 2. Distribuição dos scores das emoções dominantes
    plt.subplot(1, 2, 2)
    plt.hist(scores_dominantes, bins=20, color='lightcoral', alpha=0.7)
    plt.title('Distribuição dos Scores das Emoções Dominantes')
    plt.xlabel('Score da Emoção')
    plt.ylabel('Frequência')
    
    plt.tight_layout()
    plt.show()
    
    # 3. Estatísticas das emoções
    print("\n=== ESTATÍSTICAS DAS EMOÇÕES ===")
    print(f"Total de hinos analisados: {len(hinos_analise)}")
    print(f"Hinos com erro no processamento: {emocoes_dominantes.count('unknown')}")
    print(f"Score médio das emoções dominantes: {np.mean([s for s in scores_dominantes if s > 0]):.3f}")
    
    print("\nTop 10 emoções mais frequentes:")
    for i, (emocao, count) in enumerate(emocao_counts.head(10).items(), 1):
        percentage = (count / len(hinos_analise)) * 100
        print(f"{i:2d}. {emocao:15s}: {count:3d} hinos ({percentage:4.1f}%)")
    
    # 4. Exemplos de hinos por emoção dominante
    print("\n=== EXEMPLOS POR EMOÇÃO ===")
    for emocao in emocao_counts.head(5).index:
        hinos_emocao = hinos_analise[hinos_analise["emocao_dominante"] == emocao]
        exemplo = hinos_emocao.iloc[0]
        print(f"\n{emocao.upper()} (score: {exemplo['score_dominante']:.3f}):")
        print(f"  Hino: {exemplo['nome']}")
        print(f"  Texto: {exemplo['texto_limpo'][:100]}...")
        
    print(f"\nProcessamento concluído! Dados salvos com análise de emoções.")

In [ ]:
# Visualização detalhada: Heatmap de emoções por hino
import seaborn as sns
import matplotlib.pyplot as plt

# Verificar se os dados foram processados
if "emocoes" not in hinos_analise.columns:
    print("Execute as células anteriores primeiro para processar as emoções!")
else:
    print("Criando heatmap de emoções...")
    
    # Expandir as emoções em colunas numéricas para uma amostra
    # (usar todos os hinos seria muito denso para visualizar)
    amostra_viz = hinos_analise.head(20).copy()  # Primeiros 20 hinos
    
    # Criar dataframe com as emoções expandidas
    emocoes_expandidas = []
    nomes_hinos = []
    
    for idx, row in amostra_viz.iterrows():
        if row["emocoes"]:
            emocoes_expandidas.append(row["emocoes"])
            nomes_hinos.append(row["nome"][:30] + "...")  # Truncar nomes longos
        
    if emocoes_expandidas:
        emo_df = pd.DataFrame(emocoes_expandidas, index=nomes_hinos)
        
        # Selecionar apenas as emoções mais comuns para visualização
        emocoes_principais = emo_df.mean().nlargest(10).index
        emo_df_filtrado = emo_df[emocoes_principais]
        
        plt.figure(figsize=(12, 8))
        sns.heatmap(emo_df_filtrado, annot=True, cmap="YlOrRd", fmt=".2f", 
                   cbar_kws={'label': 'Score da Emoção'})
        plt.title("Distribuição de Emoções por Hino\n(Amostra dos primeiros 20 hinos)")
        plt.xlabel("Emoção")
        plt.ylabel("Hino")
        plt.xticks(rotation=45)
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
        
        print(f"Heatmap criado com {len(emo_df_filtrado)} hinos e {len(emocoes_principais)} emoções principais.")
        print(f"Emoções visualizadas: {list(emocoes_principais)}")
    else:
        print("Nenhuma emoção foi processada com sucesso. Verifique os dados.")

## ✅ Problema Resolvido!

### O que estava causando o erro:

1. **Parâmetro depreciado**: `return_all_scores=True` foi substituído por `top_k=None`
2. **Limite de tokens**: O modelo BERT tem limite de 512 tokens, mas alguns textos dos hinos eram muito longos (até 2250 caracteres)
3. **Formato do resultado**: O classificador retorna uma lista de listas, não uma lista simples
4. **Falta de tratamento de erro**: Sem proteção para textos problemáticos

### Correções aplicadas:

1. ✅ **Configuração correta do pipeline**: Substituído `return_all_scores` por `top_k=None`
2. ✅ **Truncamento automático**: Adicionado `max_length=512` e `truncation=True`
3. ✅ **Função de truncamento**: Criada função específica para lidar com textos longos
4. ✅ **Tratamento de erros**: Adicionado try/catch para textos problemáticos
5. ✅ **Correção do formato**: Ajustado para lidar com `resultado[0]` (lista de listas)
6. ✅ **Processamento em lotes**: Para mostrar progresso durante o processamento
7. ✅ **Visualizações melhoradas**: Heatmap e análises estatísticas

### Como usar:

1. **Execute a primeira célula** para configurar o modelo e testar com amostra
2. **Execute a segunda célula** para processar todos os 795 hinos (demora alguns minutos)
3. **Execute a terceira célula** para ver análises e estatísticas
4. **Execute a quarta célula** para visualizar o heatmap de emoções

Os resultados serão salvos em `../assets/hinos_analise_com_emocoes.pkl`.